In [64]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

In [65]:
url = 'https://boards.4chan.org/pol/catalog.json'

def get_data(url):

    response = requests.get(url)
    response = response.json()

    data = []
    for items in response:
        page = items['page']
        threads = items['threads']
        for thread in threads:
            keys_to_extract = ['no', 'id', 'sub', 'com', 'replies', 'time', 'last_modified']
            thread_data = {key: thread.get(key) for key in keys_to_extract}
            data.append(thread_data)
    
    return data

df = pd.DataFrame(get_data(url))

In [66]:
df.fillna('', inplace=True)
df['subcom'] = df['sub'] + ' ' + df['com']
df = df[['no', 'id', 'subcom', 'replies', 'time', 'last_modified']]

In [67]:
def extract_text(subcom):
    soup = BeautifulSoup(subcom, 'html.parser')
    extracted_text = soup.get_text()
    return extracted_text

df['extracted_text'] = df['subcom'].apply(extract_text)

C:\Users\HangWei\AppData\Local\Temp\ipykernel_28604\4024080341.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(subcom, 'html.parser')


In [68]:
cols = ['no', 'id', 'extracted_text', 'replies', 'time', 'last_modified']
df[cols].sort_values(by='replies', ascending=False)

,no,id,extracted_text,replies,time,last_modified
174,434305997,rpPRrCLt,aus/pol/ - ogre edition >https://archive.today...,320,1689236845,1689247193
132,434304820,uiKD+aUY,No more warnings Putin played nice and you lau...,317,1689235500,1689247289
197,434281792,vb/ib/oM,Why do people hate this movie? All my friends...,306,1689215380,1689247194
81,434282501,yh+Tns2d,"Why can’t people accept that God is in part, ...",296,1689215884,1689247129
15,434309736,vmG38zB/,/chug/ - Comfy Happening in Ukraine General #1...,283,1689240810,1689247286
...,...,...,...,...,...,...
168,434314789,uiHo4Cwh,"An old joke with a pol twist >A Jew, A Muslim,...",0,1689246259,1689246259
47,434315776,lJE1LkKf,NPNW :) Hey So basically I'm just not gonna do...,0,1689247231,1689247231
146,434315106,PJlWdHbS,Which one white man? One of these two things i...,0,1689246540,1689246540
150,434315081,pdXTsNuM,White jewish men are lazy,0,1689246516,1689246516


In [69]:
df.to_csv('../../text.csv')